<a href="https://colab.research.google.com/github/Blane187/UVR5-NO-UI/blob/main/UVR5_NO_UI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **🎵 UVR5 NO UI 🎵**

Colab created by [Not Eddy (Spanish Mod)](http://discord.com/users/274566299349155851) in [AI HUB](https://discord.gg/aihub) server.

<br>**This is a CLI type colab created based on the [beveradb's python-audio-separator](https://github.com/karaokenerds/python-audio-separator) and [jarredou's MVSEP-MDX23-Colab_v2](https://github.com/jarredou/MVSEP-MDX23-Colab_v2) repositories.<br>
If you liked this colab you can star my repo on [GitHub](https://github.com/Eddycrack864/UVR5-NO-UI)**.<br>

<br>You can donate to the original UVR5 project here:<br>
[!["Buy Me A Coffee"](https://www.buymeacoffee.com/assets/img/custom_images/orange_img.png)](https://www.buymeacoffee.com/uvr5)

In [ ]:
#@title # **Installation**
#@markdown # This takes around 3 - 5 minutes
from google.colab import drive
from IPython.display import clear_output
import subprocess
import codecs

gitrepo = codecs.decode('uggcf://tvguho.pbz/Rqqlpenpx864/ZIFRC-ZQK23-Pbyno_i2.tvg','rot_13')
dir = codecs.decode('/pbagrag/ZIFRC-ZQK23-Pbyno_i2','rot_13')

print('[+] Installation please wait....')

subprocess.run(["pip", "install", "audio-separator[gpu]==0.16.2"])

!git clone $gitrepo --quiet

%cd $dir

subprocess.run(["pip", "install", "yt_dlp"])

subprocess.run(["pip", "install", "-r", "requirements.txt"])

subprocess.run(["pip", "install", "demucs", "&>", "/dev/null"])

subprocess.run(["python", "-m", "pip", "install", "ort-nightly-gpu", "--index-url=https://aiinfra.pkgs.visualstudio.com/PublicPackages/_packaging/ort-cuda-12-nightly/pypi/simple/"])


drive.mount('/content/drive')






clear_output()
print('[~] Installation done !')

# **Select Separation method:**

### ***BS-Roformer***

In [ ]:
#@markdown #**Separation! BS-Roformer Only**
from pathlib import Path
import glob

#@markdown Input path for audio files:
input_folder = "/content/drive/MyDrive/Separar" #@param {type:"string"}
#@markdown Output path for audio files:
output_folder = "/content/drive/MyDrive/Vocales" #@param {type:"string"}
#@markdown Select the model:
model = 'BS-Roformer-Viperx-1297' #@param ["BS-Roformer-Viperx-1297", "BS-Roformer-Viperx-1296"]
#@markdown Select the output format:
format = "wav 32-bit FLOAT" #@param ["wav 16-bit PCM", "wav 32-bit FLOAT", "flac"]
weight = 10

use_BSRoformer =  '--use_BSRoformer'
use_InstVoc = '--use_InstVoc' #forced use
vocals_only = '--vocals_only'

dict1 = {
    'BS-Roformer-Viperx-1297': 'ep_317_1297',
    'BS-Roformer-Viperx-1296': 'ep_368_1296'
}
BSRoformer_model = dict1[model]

dict2 = {
    'wav 16-bit PCM': 'PCM_16',
    'wav 32-bit FLOAT': 'FLOAT',
    'flac': 'FLAC'
}
output_format = dict2[format]

file_paths = sorted(glob.glob(input_folder + "/*"))[:]
input_audio_args = ' '.join([f'"{path}"' for path in file_paths])
Path(output_folder).mkdir(parents=True, exist_ok=True)
!python inference.py \
    --input_audio {input_audio_args} \
    --large_gpu \
    --BSRoformer_model {BSRoformer_model} \
    --weight_BSRoformer {weight} \
    --output_format {output_format} \
    --output_folder "{output_folder}" \
    {vocals_only} \
    {use_InstVoc} \
    {use_BSRoformer}

# jarredou's code

### ***MDX23C***

In [ ]:
#@markdown #**Separation! MDX23C Only**
import os
import subprocess

def uvr_cli(input_folder, output_folder, extensions, output_format, model, segment_size, overlap):
    found_files = []

    for audio_files in os.listdir(input_folder):
        if audio_files.endswith(extensions):
            found_files.append(audio_files)

    total_files = len(found_files)

    if total_files == 0:
        print("No valid audio files found.")
    else:
        print(f"{total_files} audio files found")

        found_files.sort()

        for audio_files in found_files:
            file_path = os.path.join(input_folder, audio_files)
            prompt = ["audio-separator", file_path, "-m", f"{model}", f"--output_dir={output_folder}", f"--output_format={output_format}", "--normalization=0.9", f"--mdxc_segment_size={segment_size}", f"--mdxc_overlap={overlap}"]
            print(f"Processing file: {audio_files}")
            subprocess.run(prompt)

#@markdown Input path for audio files:
input_folder = "/content/drive/MyDrive/Separar" #@param {type:"string"}
#@markdown Output path for audio files:
output_folder = "/content/drive/MyDrive/Vocales" #@param {type:"string"}
#@markdown Select the model:
model = "MDX23C-8KFFT-InstVoc_HQ_2.ckpt" #@param ["MDX23C_D1581.ckpt", "MDX23C-8KFFT-InstVoc_HQ.ckpt", "MDX23C-8KFFT-InstVoc_HQ_2.ckpt"]
#@markdown Select the output format:
output_format = "wav" #@param ["wav", "flac", "mp3"]
#@markdown Larger consumes more resources, but may give better results.
segment_size = 256 #@param {type:"slider", min:32, max:4000, step:32}
#@markdown Amount of overlap between prediction windows.
overlap = 8 #@param {type:"slider", min:2, max:50, step:1}
extensions = (".mp3", ".wav", ".flac")

uvr_cli(input_folder, output_folder, extensions, output_format, model, segment_size, overlap)

### ***MDX-NET***

In [ ]:
#@markdown #**Separation! MDX-NET Only**
import os
import subprocess

def uvr_cli(input_folder, output_folder, extensions, output_format, model, segment_size, overlap, denoise):
    found_files = []

    for audio_files in os.listdir(input_folder):
        if audio_files.endswith(extensions):
            found_files.append(audio_files)

    total_files = len(found_files)

    if total_files == 0:
        print("No valid audio files found.")
    else:
        print(f"{total_files} audio files found")

        found_files.sort()

        for audio_files in found_files:
            file_path = os.path.join(input_folder, audio_files)
            prompt = ["audio-separator", file_path, "-m", f"{model}", f"--output_dir={output_folder}", f"--output_format={output_format}", "--normalization=0.9", f"--mdx_segment_size={segment_size}", f"--mdx_overlap={overlap}"]
            if denoise:
                prompt.append("--mdx_enable_denoise")
            print(f"Processing file: {audio_files}")
            subprocess.run(prompt)

#@markdown Input path for audio files:
input_folder = "/content/drive/MyDrive/audio/yukan_no_shonen/" #@param {type:"string"}
#@markdown Output path for audio files:
output_folder = "/content/drive/MyDrive/Vocales" #@param {type:"string"}
#@markdown Select the model:
model = "UVR-MDX-NET-Inst_HQ_4.onnx" #@param ["UVR-MDX-NET-Inst_full_292.onnx", "UVR-MDX-NET_Inst_187_beta.onnx", "UVR-MDX-NET_Inst_82_beta.onnx", "UVR-MDX-NET_Inst_90_beta.onnx", "UVR-MDX-NET_Main_340.onnx", "UVR-MDX-NET_Main_390.onnx", "UVR-MDX-NET_Main_406.onnx", "UVR-MDX-NET_Main_427.onnx", "UVR-MDX-NET_Main_438.onnx", "UVR-MDX-NET-Inst_HQ_1.onnx", "UVR-MDX-NET-Inst_HQ_2.onnx", "UVR-MDX-NET-Inst_HQ_3.onnx", "UVR-MDX-NET-Inst_HQ_4.onnx", "UVR_MDXNET_Main.onnx", "UVR-MDX-NET-Inst_Main.onnx", "UVR_MDXNET_1_9703.onnx", "UVR_MDXNET_2_9682.onnx", "UVR_MDXNET_3_9662.onnx", "UVR-MDX-NET-Inst_1.onnx", "UVR-MDX-NET-Inst_2.onnx", "UVR-MDX-NET-Inst_3.onnx", "UVR_MDXNET_KARA.onnx", "UVR_MDXNET_KARA_2.onnx", "UVR_MDXNET_9482.onnx", "UVR-MDX-NET-Voc_FT.onnx", "Kim_Vocal_1.onnx", "Kim_Vocal_2.onnx", "Kim_Inst.onnx", "Reverb_HQ_By_FoxJoy.onnx", "UVR-MDX-NET_Crowd_HQ_1.onnx", "kuielab_a_vocals.onnx", "kuielab_a_other.onnx", "kuielab_a_bass.onnx", "kuielab_a_drums.onnx", "kuielab_b_vocals.onnx", "kuielab_b_other.onnx", "kuielab_b_bass.onnx", "kuielab_b_drums.onnx"]
#@markdown Select the output format:
output_format = "wav" #@param ["wav", "flac", "mp3"]
#@markdown Larger consumes more resources, but may give better results.
segment_size = 256 #@param {type:"slider", min:32, max:4000, step:32}
#@markdown Amount of overlap between prediction windows.
overlap = "0.25" #@param ["0.25", "0.5", "0.75", "0.99"]
#@markdown Enable denoising during separation.
denoise = True #@param {type:"boolean"}
extensions = (".mp3", ".wav", ".flac")

uvr_cli(input_folder, output_folder, extensions, output_format, model, segment_size, overlap, denoise)

### ***VR ARCH***

In [ ]:
#@markdown #**Separation! VR ARCH Only**
import os
import subprocess

def uvr_cli(input_folder, output_folder, extensions, output_format, model, window_size, aggression, tta):
    found_files = []

    for audio_files in os.listdir(input_folder):
        if audio_files.endswith(extensions):
            found_files.append(audio_files)

    total_files = len(found_files)

    if total_files == 0:
        print("No valid audio files found.")
    else:
        print(f"{total_files} audio files found")

        found_files.sort()

        for audio_files in found_files:
            file_path = os.path.join(input_folder, audio_files)
            prompt = ["audio-separator", file_path, "-m", f"{model}", f"--output_dir={output_folder}", f"--output_format={output_format}", "--normalization=0.9", f"--vr_window_size={window_size}", f"--vr_aggression={aggression}"]
            if tta:
                prompt.append("--vr_enable_tta")
            print(f"Processing file: {audio_files}")
            subprocess.run(prompt)

#@markdown Input path for audio files:
input_folder = "/content/drive/MyDrive/Separar" #@param {type:"string"}
#@markdown Output path for audio files:
output_folder = "/content/drive/MyDrive/Vocales" #@param {type:"string"}
#@markdown Select the model:
model = "UVR-DeEcho-DeReverb.pth" #@param ["1_HP-UVR.pth", "2_HP-UVR.pth", "3_HP-Vocal-UVR.pth", "4_HP-Vocal-UVR.pth", "5_HP-Karaoke-UVR.pth", "6_HP-Karaoke-UVR.pth", "7_HP2-UVR.pth", "8_HP2-UVR.pth", "9_HP2-UVR.pth", "10_SP-UVR-2B-32000-1.pth", "11_SP-UVR-2B-32000-2.pth", "12_SP-UVR-3B-44100.pth", "13_SP-UVR-4B-44100-1.pth", "14_SP-UVR-4B-44100-2.pth", "15_SP-UVR-MID-44100-1.pth", "16_SP-UVR-MID-44100-2.pth", "17_HP-Wind_Inst-UVR.pth", "UVR-De-Echo-Aggressive.pth", "UVR-De-Echo-Normal.pth", "UVR-DeEcho-DeReverb.pth", "UVR-DeNoise-Lite.pth", "UVR-DeNoise.pth", "UVR-BVE-4B_SN-44100-1.pth", "MGM_HIGHEND_v4.pth", "MGM_LOWEND_A_v4.pth", "MGM_LOWEND_B_v4.pth", "MGM_MAIN_v4.pth"]
#@markdown Select the output format:
output_format = "wav" #@param ["wav", "flac", "mp3"]
#@markdown Lower window size yield a higher output quality, but will take longer to process.
window_size = "320" #@param ["320", "512", "1024"]
#@markdown Intensity of primary stem extraction.
aggression = 5 #@param {type:"slider", min:0, max:100, step:1}
#@markdown Enable Test-Time-Augmentation; slow but improves quality.
tta = True #@param {type:"boolean"}
extensions = (".mp3", ".wav", ".flac")

uvr_cli(input_folder, output_folder, extensions, output_format, model, window_size, aggression, tta)

### ***demucs***

In [ ]:
#@markdown #**Separation! Demucs Only**
import os
import subprocess

def uvr_cli(input_folder, output_folder, extensions, output_format, model):
    found_files = []

    for audio_files in os.listdir(input_folder):
        if audio_files.endswith(extensions):
            found_files.append(audio_files)

    total_files = len(found_files)

    if total_files == 0:
        print("No valid audio files found.")
    else:
        print(f"{total_files} audio files found")

        found_files.sort()

        for audio_files in found_files:
            file_path = os.path.join(input_folder, audio_files)
            prompt = ["audio-separator", file_path, "-m", f"{model}", f"--output_dir={output_folder}", f"--output_format={output_format}", "--normalization=0.9"]
            print(f"Processing file: {audio_files}")
            subprocess.run(prompt)

#@markdown Input path for audio files:
input_folder = "/content/drive/MyDrive/Separar" #@param {type:"string"}
#@markdown Output path for audio files:
output_folder = "/content/drive/MyDrive/Vocales" #@param {type:"string"}
#@markdown Select the model:
model = "htdemucs_ft.yaml" #@param ["htdemucs_ft.yaml", "htdemucs.yaml", "hdemucs_mmi.yaml", "htdemucs_6s.yaml"]
#@markdown Select the output format:
output_format = "wav" #@param ["wav", "flac", "mp3"]
extensions = (".mp3", ".wav", ".flac")

uvr_cli(input_folder, output_folder, extensions, output_format, model)

# Extra

extra selection  by Blane187

In [ ]:
#@markdown # yt_dlp

import yt_dlp

url = 'https://www.youtube.com/watch?v=BaW_jenozKc #@param {type:"string"}

ydl_opts = {
    'format': 'wav/bestaudio/best'
    "outtmpl": 'youtubeaudio/%(title)s.%(ext)s',  # this is where you can edit how you'd like the filenames to be formatted
    # ℹ️ See help(yt_dlp.postprocessor) for a list of available Postprocessors and their arguments
    'postprocessors': [{  # Extract audio using ffmpeg
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'wav',
    }]
}

with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    error_code = ydl.download(url)